In [1]:
import pandas as pd
import numpy as np
from collections import Counter

In [2]:
#bara khondan diabet
information_data = pd.read_csv("diabetes.csv")
information_data.head()
information_data.columns[1]
information_data.info()
information_data1= information_data

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 768 entries, 0 to 767
Data columns (total 9 columns):
 #   Column                    Non-Null Count  Dtype  
---  ------                    --------------  -----  
 0   Pregnancies               768 non-null    int64  
 1   Glucose                   768 non-null    int64  
 2   BloodPressure             768 non-null    int64  
 3   SkinThickness             768 non-null    int64  
 4   Insulin                   768 non-null    int64  
 5   BMI                       768 non-null    float64
 6   DiabetesPedigreeFunction  768 non-null    float64
 7   Age                       768 non-null    int64  
 8   Outcome                   768 non-null    int64  
dtypes: float64(2), int64(7)
memory usage: 54.1 KB


In [3]:
x=information_data.drop(['Outcome'], axis=1)
x
#data diabet ra mirizim be ger az outcom

,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age
0,6,148,72,35,0,33.6,0.627,50
1,1,85,66,29,0,26.6,0.351,31
2,8,183,64,0,0,23.3,0.672,32
3,1,89,66,23,94,28.1,0.167,21
4,0,137,40,35,168,43.1,2.288,33
...,...,...,...,...,...,...,...,...
763,10,101,76,48,180,32.9,0.171,63
764,2,122,70,27,0,36.8,0.340,27
765,5,121,72,23,112,26.2,0.245,30
766,1,126,60,0,0,30.1,0.349,47


In [4]:
y = information_data.Outcome
y
#outcom ra inja mirizim

0      1
1      0
2      1
3      0
4      1
      ..
763    0
764    0
765    0
766    1
767    0
Name: Outcome, Length: 768, dtype: int64

In [5]:

from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=1)
#0.2 az data ra be train midahad 0.8 ra be test midahad bad aval train mikonim bad test

In [6]:
class Node:
    def __init__(self, feature=None, threshold=None, children = None, gain = None,*,value= None) -> None:
        self.feature = feature
        self.threshold = threshold#?
        self.children = children
        self.value=value
        self.gain = gain

    def is_leaf_node(self):
        return self.value is not None


In [7]:

class DecisionTree:
    def __init__(self, min_samples_split=2, max_depth=3, childSize = 4, n_features=None):
        self.min_samples_split=min_samples_split
        self.max_depth=max_depth
        self.n_features=n_features
        self.root=None
        self.childSize = childSize

    def fit(self, X, y):
        #grow tree ra seda mizanad
        self.n_features = X.shape[1] if not self.n_features else min(X.shape[1],self.n_features)
        print(X.shape[1])
        self.root = self._grow_tree(X, y)

    def _grow_tree(self, X, y, depth=0):
        #drakht ra bast midahad va do tabe mohem darad
        print(f'depth {depth}')
        n_samples, n_feats = X.shape
        n_labels = len(np.unique(y))

        if (depth>=self.max_depth or n_labels==1 or n_samples < self.min_samples_split):
            leaf_value = self._most_common_label(y)
            return Node(value=leaf_value)

        feat_idxs = np.random.choice(n_feats, self.n_features, replace=False)
        print("before best split")

        best_feature, best_thresh = self._best_split(X, y, feat_idxs)
        print("before split")
        child_idxs = self._split(X[: ,best_feature], best_thresh)
        child_idxs = np.array(child_idxs)
        children =[]

        print("before grow tree")
        print(f'X:{X}')
        print(f'y:{y}')
        for i in range(len(child_idxs)):
            print(f'child:{child_idxs}')
            if y[child_idxs[i]] == []:
                print("child_idx is empty")
            children.append(self._grow_tree( X[child_idxs[i],:],y[child_idxs[i]],depth+1))

        node = Node(best_feature, best_thresh, children)
        print(f'feature: {node.feature} children: {node.children} ')
        
        return node

    def _best_split(self, X, y, feat_idxs):
        #thresh hold ra be dast miavarad va best ficher
        np.random.seed()
        split_idx = None
        split_threshold = []
        best_gains = -1
        for feat_idx in feat_idxs:
            threshold = []
            X_column = X[:,feat_idx]
            thresholds = np.unique(X_column)
            temp = np.unique(thresholds)
            temp.sort()
            length = temp.size
            everyPart = length /  self.childSize
            for i in range(self.childSize - 1):
                num = int((i+1)* everyPart)
                threshold.append(temp[num])
            gain = self._information_gain(y, X_column, threshold)  
            if gain > best_gains:
                best_gains = gain
                split_idx = feat_idx
                split_threshold = threshold        
        return split_idx, split_threshold


    def _information_gain(self, y, X_column, threshold):
#mohasebe entropy va jam anha
        parent_entropy = self._entropy(y)

        children = self._split(X_column, threshold)
        chil = 0
        for i in children:
            chil += len(i)
        print(f'sum of {chil}')

        for i in children:
            if len(i) == 0:

                return 0
        

        n = len(y)
        n_l = []
        e_l = []
        for i in children:
            n_l.append(len(i))
        for i in children:
            e_l.append(self._entropy(y[i]))
        child_entropy = 0
        for i in range(len(children)):
            child_entropy += (n_l[i]/n) * e_l[i]

        information_gain = parent_entropy - child_entropy
        if information_gain < 0:
            print(f'len{X_column.size}')
            print(f'information gain-> parent: {parent_entropy} child: {child_entropy}')
            print(f'parentlen: {len(y)} child')
            print(f'e_l -> {e_l}')
            print(f'n_l -> {n_l}')
        return information_gain

    def _split(self, X_column, split_thresh):
#bara tagsim kardan data
        answer = []

        split_thresh.sort()
        if len(np.argwhere(X_column > split_thresh[0]).flatten()) == 0: 
            answer.append(np.argwhere(X_column < split_thresh[0]).flatten())
        else:
            answer.append(np.argwhere(X_column <= split_thresh[0]).flatten())
        count = 0
        while count < len(split_thresh) - 2:
            if len(np.argwhere(X_column > split_thresh[count]).flatten()) == 0: 
                answer.append(np.argwhere(X_column >= split_thresh[count]).flatten())

                answer.append(np.argwhere(X_column < split_thresh[count]).flatten())
                count+= 1
            else:
                answer.append(np.argwhere(X_column > split_thresh[count]).flatten())

                answer.append(np.argwhere(X_column <= split_thresh[count]).flatten())
                count+= 1
        if len(np.argwhere(X_column > split_thresh[len(split_thresh) - 1]).flatten()) == 0: 
            answer.append(np.argwhere(X_column >= split_thresh[len(split_thresh) - 1]).flatten())
        else:
            answer.append(np.argwhere(X_column > split_thresh[len(split_thresh) - 1]).flatten())

        print(f'answer: {answer}')
        print(f'X_column: {X_column}')
        print(f'split_thresh: {split_thresh}')
        return answer

    def _entropy(self, y):
        #ehtemal har kodam ra hesab mikonad va....
        hist = np.bincount(y)
        ps = hist / len(y)
        return -np.sum([p * np.log(p) for p in ps if p>0])


    def _most_common_label(self, y):
        #entekhab bishtarin
        counter = Counter(y)
        print(f'most common{y}')
        value = counter.most_common(1)[0][0]
        return value

    def predict(self, X):
        #tabe traverse tree ra seda mizanad
        return np.array([self._traverse_tree(x, self.root) for x in X])

    def _traverse_tree(self, x, node):
        #accurasy ra be ma midaha (natige nahayi)
        if node.is_leaf_node():
            return node.value
        for i, thr in enumerate(node.threshold):
            if x[node.feature] <= thr:
                return self._traverse_tree(x, node.children[i])

        return self._traverse_tree(x, node.children[len(node.children) - 1])    

In [8]:
x = x.to_numpy()
y = y.to_numpy()

In [9]:
x_train , x_test, y_train, y_test = tarin = train_test_split(x, y , test_size = 0.2, random_state = 4321)

In [10]:
clf = DecisionTree()
clf.fit(x_train, y_train)
prediction = clf.predict(x_test)
#drakht ra misazim va shro be train bad natige mide bad ba test mogayese mikone

8
depth 0
before best split
answer: [array([  0,   6,  11,  13,  17,  25,  36,  40,  43,  60,  63,  67,  72,
        77,  79,  81,  82,  89,  90,  96, 100, 105, 120, 121, 131, 132,
       135, 136, 145, 147, 165, 170, 177, 180, 183, 184, 185, 192, 194,
       197, 202, 205, 208, 212, 229, 230, 235, 249, 260, 262, 264, 266,
       276, 284, 285, 292, 298, 302, 326, 330, 334, 337, 344, 346, 350,
       356, 361, 362, 378, 380, 381, 385, 388, 389, 393, 394, 398, 406,
       409, 422, 423, 432, 433, 438, 444, 447, 451, 457, 460, 464, 465,
       469, 471, 476, 478, 481, 483, 493, 497, 500, 501, 503, 506, 509,
       516, 521, 524, 533, 534, 537, 540, 541, 542, 544, 546, 547, 549,
       552, 553, 554, 558, 561, 566, 567, 569, 574, 577, 587, 589, 594,
       599, 604, 606, 607], dtype=int64), array([  1,   2,   3,   4,   5,   7,   8,   9,  10,  12,  14,  15,  16,
        18,  19,  20,  21,  22,  23,  24,  26,  27,  28,  29,  30,  31,
        32,  33,  34,  35,  37,  38,  39,  41,  42,  44, 

C:\Users\ASUS\AppData\Local\Temp\ipykernel_7296\3639967921.py:31: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  child_idxs = np.array(child_idxs)
C:\Users\ASUS\AppData\Local\Temp\ipykernel_7296\3639967921.py:39: DeprecationWarning: elementwise comparison failed; this will raise an error in the future.
  if y[child_idxs[i]] == []:


X_column: [26.2 32.4 34.2 42.7 27.8 21.7 44.6 24.4 30.4 28.7 35.3 36.9 30.  29.7
 34.  22.5 26.6 23.2 23.9 24.2 28.2 24.4  0.  27.5 39.9 37.7 32.  42.2
 46.3 37.4 30.1 25.  37.2 35.6 37.6 38.3 23.7 33.2 23.  22.5 40.9 20.4
 32.  36.5 41.  32.  39.5 29.2 27.2 26.  31.6 25.  32.  43.5 30.4 36.8
 32.1 31.2 21.1 28.  27.3 39.  24.4 24.2 21.2 35.5 35.6 43.5 26.8 28.4
 29.3 27.7 33.6 22.4 28.7 38.  21.1 25.4 36.8 27.7 35.8 29.  38.2 26.6
 38.2 24.5 34.3 32.7  0.  30.2 24.7 27.2 32.  24.7 33.3 29.8 32.2 45.3
 27.3 27.6 25.9 30.1 29.  21.8 21.8 18.2 39.6 21.1 34.9 28.1 25.2 34.4
 26.1 19.1 30.4 33.2 37.3 55.  19.9 23.1 35.3 24.8 33.5  0.  39.8 36.8
 33.1 20.1 43.4 29.7 41.3 35.  19.6 37.7 32.  34.6 36.9 28.  31.  24.7
 37.2 35.4 33.7 25.1 24.2 27.8 32.8]
split_thresh: [25.4, 31.0, 36.5]
sum of 220
len147
information gain-> parent: 0.329544105160137 child: 0.41102212026413076
parentlen: 147 child
e_l -> [-0.0, 0.40530264656429066, -0.0, 0.5167535908014582]
n_l -> [40, 107, 40, 33]
answer: [arra

C:\Users\ASUS\AppData\Local\Temp\ipykernel_7296\3639967921.py:39: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if y[child_idxs[i]] == []:


In [11]:
def accuracy(y_test, y_pred):
    return np.sum(y_test == y_pred)/len(y_test)
#y_test va y_prediction mogayese mikonad mibine chegadr javab dorost dadim

In [12]:
acc = accuracy(y_test, prediction)
print(acc)

0.7077922077922078


In [13]:
def dfs(tree:Node, depth = 0):
    if tree != None:
        if tree.children != None:
            print(depth * '\t' + f'threshold:{tree.threshold} and feature:{information_data1.columns[tree.feature]}')
            for child in tree.children:
                dfs(child, depth+1)
        else:
            print(depth * '\t' + f'value: {tree.value}')
#derakht ra chap mikonad
            
dfs(clf.root)

threshold:[97.0, 130.0, 163.0] and feature:Glucose
	threshold:[56.0, 70.0, 85.0] and feature:BloodPressure
		threshold:[36.0, 76.0, 99.0] and feature:Insulin
			value: 0
			value: 0
			value: 0
			value: 0
		threshold:[66.0, 76.0, 88.0] and feature:BloodPressure
			value: 0
			value: 0
			value: 0
			value: 0
		threshold:[36.0, 76.0, 99.0] and feature:Insulin
			value: 0
			value: 0
			value: 0
			value: 0
		value: 0
	threshold:[26.9, 33.9, 39.4] and feature:BMI
		threshold:[58.0, 68.0, 80.0] and feature:BloodPressure
			value: 0
			value: 0
			value: 0
			value: 0
		threshold:[58.0, 75.0, 92.0] and feature:BloodPressure
			value: 0
			value: 1
			value: 0
			value: 1
		threshold:[58.0, 68.0, 80.0] and feature:BloodPressure
			value: 0
			value: 0
			value: 0
			value: 0
		threshold:[120.0, 142.0, 166.0] and feature:Glucose
			value: 0
			value: 1
			value: 0
			value: 1
	threshold:[56.0, 70.0, 85.0] and feature:BloodPressure
		threshold:[36.0, 76.0, 99.0] and feature:Insulin
			value: